In [1]:
import tensorflow as tf
from src.robot_reboot.dataset_factory import RobotRebootDataSetFactory
tf.__version__

'2.4.1'

In [2]:
cnn = tf.keras.models.load_model('model/model_0')

In [3]:
ds_factory = RobotRebootDataSetFactory(31, cnn, max_depth=20, seed=26, playouts=50)
v, p, s = ds_factory.create(locate_robot_close_goal=True, max_movements=3)

In [4]:
rows, cols, layers = 31, 31, 9
state = s.reshape(rows * cols * layers,)
state

array([0., 0., 0., ..., 0., 0., 0.])

### Serialize

In [5]:
sample = tf.train.Example(
    features = tf.train.Features(
        feature = {
            'v': tf.train.Feature(float_list=tf.train.FloatList(value=[v])),
            'p': tf.train.Feature(float_list=tf.train.FloatList(value = p)),
            's': tf.train.Feature(float_list=tf.train.FloatList(value = state))
        }
    )
)

In [6]:
with tf.io.TFRecordWriter('tf_record_data') as f:
    f.write(sample.SerializeToString())

### Deserialize

In [7]:
feature_desc = {
    'v': tf.io.FixedLenFeature([], tf.float32),
    'p': tf.io.VarLenFeature(tf.float32),
    's': tf.io.VarLenFeature(tf.float32)
}

In [25]:
import numpy as np
for serialized in tf.data.TFRecordDataset(['tf_record_data']):
    parsed = tf.io.parse_single_example(serialized, feature_desc)
    st= tf.sparse.to_dense(parsed['s'])
    t = tf.reshape(st, (1, rows, cols, layers))
    print(cnn.predict(t))

[array([[-0.03681714]], dtype=float32), array([[ 0.01798075, -0.00481379,  0.04521103, -0.00659672,  0.03973615,
         0.02471904, -0.00596441,  0.00939979,  0.00366622, -0.03947062,
         0.01523161,  0.01026706,  0.00846202,  0.04074233, -0.00917254,
         0.0018367 ]], dtype=float32)]


### Train NN

In [26]:
def read_tfrecord(example):
    feature_desc = {
        'v': tf.io.FixedLenFeature([], tf.float32),
        'p': tf.io.VarLenFeature(tf.float32),
        's': tf.io.VarLenFeature(tf.float32)
    }
    sample = tf.io.parse_single_example(example, feature_desc)
    y = {'v':sample['v'], 'p': tf.sparse.to_dense(sample['p'])}
    return tf.reshape(tf.sparse.to_dense(parsed['s']), (1,rows,cols, layers)), y

In [27]:
ds = tf.data.TFRecordDataset(['tf_record_data'])
ds = ds.map(read_tfrecord)

In [28]:
cnn.fit(ds)

ValueError: in user code:

    /Users/danielavelasquez/mlearning/robot-reboot/src/robotenv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/danielavelasquez/mlearning/robot-reboot/src/robotenv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/danielavelasquez/mlearning/robot-reboot/src/robotenv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/danielavelasquez/mlearning/robot-reboot/src/robotenv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/danielavelasquez/mlearning/robot-reboot/src/robotenv/lib/python3.7/site-pa